# 국장 & 미장 100위 크롤링

In [158]:
#필요한 라이브러리
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [159]:
# 크롤링할 대상 URL (한국 시가총액 상위 기업 목록)
url = "https://msg.soledot.com/finance/fo/sisemarketsumlist.sd#google_vignette"
# 사이트에 요청 보내고 HTML 받기
response = requests.get(url)
response

<Response [200]>

In [160]:
# 받은 HTML을 BeautifulSoup 객체로 파싱
soup = BeautifulSoup(response.text, "html.parser")
soup


<!--* TagLib -->
<!-- TagLib -->
<!--* Var  -->
<!-- Var -->
<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="soledot" name="author"/>
<meta content="실시간 시가총액 100위 종목정보를 제공하는 페이지입니다." name="description"/>
<meta content="시가총액,주식,상위거래,상위종목,실시간주식,실시간주가,코스피,코스닥,국내증시,상한가,하한가,종목순위,매도호가,매수호가,시가총액,현재가,전일비,등락률,종목명" name="keywords">
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="#0c2770" name="theme-color"/>
<meta content="실시간 시가총액 100위 종목정보 - 무색계 블로그" property="og:title"/>
<meta content="실시간 시가총액 100위 종목정보를 제공하는 페이지입니다." property="og:description"/>
<meta content="https://msg.soledot.com/data/img/etc/msg.png" property="og:image"/>
<meta content="https://msg.soledot.com/finance/fo/sisemarketsumlist.sd" property="og:url"/>
<meta content="website" property="og:type"/>
<link href="https://msg.soledot.com/finance/fo/sisemarketsumlist.sd" rel="canonical"/>
<title>실시간 시가총액 100위 종

In [161]:
# 테이블에서 데이터 추출
rows = soup.select("table tbody tr")
rows



[<tr>
 <td class="text-center">1</td>
 <td class="text-left">삼성전자</td>
 <td class="text-right">
 																	63,800
 																</td>
 <td class="text-right text-danger">
 																	3,000
 																</td>
 <td class="text-right text-danger">+4.93%</td>
 <td class="text-right">
 																	100
 																</td>
 <td class="text-right">
 																	3,776,729
 																</td>
 <td class="text-right">
 																	5,919,638
 																</td>
 <td class="text-right">
 																	49.64
 																</td>
 <td class="text-right">
 																	31,792,165
 																</td>
 <td class="text-right">12.36</td>
 <td class="text-right">9.03</td>
 </tr>,
 <tr>
 <td class="text-center">2</td>
 <td class="text-left">SK하이닉스</td>
 <td class="text-right">
 																	278,500
 																</td>
 <td class="text-right text-primary">
 																	500
 																</td>
 <td class="text-right text-primary">-0.18%</t

In [162]:
url = "https://companiesmarketcap.com/south-korea/largest-companies-in-south-korea-by-market-cap/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# 테이블의 tbody 안 tr 선택
rows = soup.select("table tbody tr")

data = []

for row in rows:
    rank_tag = row.select_one("td.rank-td")
    name_tag = row.select_one("div.company-name")
    ticker_tag = row.select_one("div.company-code")
    tds = row.select("td")
    
    if rank_tag and name_tag and ticker_tag and len(tds) >= 6:
        rank = rank_tag.text.strip()
        name = name_tag.text.strip()
        ticker = ticker_tag.text.strip()
        market_cap = tds[3].text.strip()
        price = tds[4].text.strip()
        today = tds[5].text.strip()
        
        data.append({
            "Rank": rank,
            "Ticker": ticker,
            "Name": name,
            "Market Cap": market_cap,
            "Price": price,
            "Today": today
        })

df_kr = pd.DataFrame(data)
print(df_kr)

   Rank     Ticker                Name Market Cap    Price  Today
0     1  005930.KS             Samsung  $309.70 B   $46.93  4.93%
1     2  000660.KS            SK Hynix  $141.40 B  $204.84  0.18%
2     3  373220.KS  LG Energy Solution   $54.81 B  $234.26  5.29%
3     4  207940.KS   Samsung Biologics   $54.65 B  $767.87  3.47%
4     5      HYMTF             Hyundai   $38.66 B   $53.00  0.00%
..  ...        ...                 ...        ...      ...    ...
95   96  000080.KS           HiteJinro    $1.06 B   $15.52  0.96%
96   97  073240.KS          Kumho Tire    $1.03 B    $3.60  3.49%
97   98  007310.KS              Ottogi    $1.03 B  $299.72  0.24%
98   99  137310.KS        SD BioSensor    $0.92 B    $7.71  1.65%
99  100  192080.KS        DoubleUGames    $0.84 B   $42.51  0.34%

[100 rows x 6 columns]


In [163]:
df_kr['Rank'].value_counts()

Rank
1      1
2      1
3      1
4      1
5      1
      ..
96     1
97     1
98     1
99     1
100    1
Name: count, Length: 100, dtype: int64

In [164]:
df_kr[df_kr['Rank'] == 1]

,Rank,Ticker,Name,Market Cap,Price,Today


- 코스피와 코스닥 TOP100이 전부 크롤링된것을 확인 -> 코스피만 필요하기 때문에 코스닥 데이터는 전처리 진행

In [165]:
df_kr = df_kr[:100]
df_kr

,Rank,Ticker,Name,Market Cap,Price,Today
0,1,005930.KS,Samsung,$309.70 B,$46.93,4.93%
1,2,000660.KS,SK Hynix,$141.40 B,$204.84,0.18%
2,3,373220.KS,LG Energy Solution,$54.81 B,$234.26,5.29%
3,4,207940.KS,Samsung Biologics,$54.65 B,$767.87,3.47%
4,5,HYMTF,Hyundai,$38.66 B,$53.00,0.00%
...,...,...,...,...,...,...
95,96,000080.KS,HiteJinro,$1.06 B,$15.52,0.96%
96,97,073240.KS,Kumho Tire,$1.03 B,$3.60,3.49%
97,98,007310.KS,Ottogi,$1.03 B,$299.72,0.24%
98,99,137310.KS,SD BioSensor,$0.92 B,$7.71,1.65%


# 미국 주식장

In [166]:
url1 = 'https://companiesmarketcap.com/'

response = requests.get(url1)
soup = BeautifulSoup(response.text, "html.parser")

rows = soup.select("table tbody tr")

data = []

for row in rows:
    rank_tag = row.select_one("td.rank-td")
    name_tag = row.select_one("div.company-name")
    ticker_tag = row.select_one("div.company-code")
    tds = row.select("td")
    
    if rank_tag and name_tag and len(tds) >= 6:
        rank = rank_tag.text.strip()
        name = name_tag.text.strip()
        Ticker = ticker_tag.text.strip()
        market_cap = tds[3].text.strip()
        price = tds[4].text.strip()
        today = tds[5].text.strip()
        
        data.append({
            "Rank": rank,
            "Ticker" : Ticker,
            "Name": name,
            "Market Cap": market_cap,
            "Price": price,
            "Today": today

        })

df_gb = pd.DataFrame(data)

print(df_gb)  # 상위 10개 출력


   Rank  Ticker                      Name Market Cap    Price  Today
0     1    NVDA                    NVIDIA   $3.834 T  $157.25  2.58%
1     2    MSFT                 Microsoft   $3.650 T  $491.09  0.20%
2     3    AAPL                     Apple   $3.172 T  $212.44  2.22%
3     4    AMZN                    Amazon   $2.334 T  $219.92  0.24%
4     5    GOOG         Alphabet (Google)   $2.173 T  $179.76  1.61%
..  ...     ...                       ...        ...      ...    ...
95   96       C                 Citigroup  $162.04 B   $86.76  0.57%
96   97  AIR.PA                    Airbus  $162.01 B  $205.78  0.46%
97   98     TMO  Thermo Fisher Scientific  $161.47 B  $427.76  1.49%
98   99    SPGI                S&P Global  $161.20 B  $525.64  0.70%
99  100  ITX.MC                   Inditex  $160.92 B   $51.66  0.48%

[100 rows x 6 columns]


- 미장이 미국주시장만 뜻하는건지 아니면 전세계 주식장을 뜻하는지 애매해서 url만 바꾸면 되게 설정
- 미국 주식 url : 'https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/' 


### 야후 API를 이용하여 두달간의 주가 저장 - 국내

In [167]:
#!pip install yfinance

In [168]:
import yfinance as yf

In [169]:
#어떤식으로 나오나 확인하는 코드 
#ticker = yf.Ticker("219480.KS")  # 삼성전자
#hist = ticker.history(period="2mo", interval="4h")  # 최근 1개월, 4시간 간격
#hist


In [170]:
#리스트를 만들어 데이터프레임을 추가하는 코드
korea_history = []
for i in range(len(df_kr)):
    name = df_kr['Name'][i]
    Ticker = df_kr['Ticker'][i]
    ticker = yf.Ticker(Ticker)
    hist = ticker.history(period="2mo", interval="4h")
    hist['name'] = name
    hist['Ticker'] = Ticker
    korea_history.append(hist)
korea_history

[                              Open     High      Low    Close    Volume  \
 Datetime                                                                  
 2025-05-07 09:00:00+09:00  54500.0  55000.0  54300.0  54300.0   4209646   
 2025-05-07 13:00:00+09:00  54350.0  54500.0  54300.0  54450.0   2880602   
 2025-05-08 09:00:00+09:00  55100.0  55500.0  54700.0  54900.0   7738123   
 2025-05-08 13:00:00+09:00  54900.0  55000.0  54600.0  54650.0   2831741   
 2025-05-09 09:00:00+09:00  54700.0  54900.0  54400.0  54650.0   4073238   
 ...                            ...      ...      ...      ...       ...   
 2025-07-01 13:00:00+09:00  60600.0  60600.0  60300.0  60500.0   2667508   
 2025-07-02 09:00:00+09:00  60200.0  60800.0  59800.0  60700.0  10224354   
 2025-07-02 13:00:00+09:00  60600.0  60800.0  60500.0  60550.0   2536969   
 2025-07-03 09:00:00+09:00  61300.0  62700.0  61100.0  62550.0  18238557   
 2025-07-03 13:00:00+09:00  62550.0  63500.0  62550.0  63500.0   8193679   
 
          

In [171]:
korea_df = pd.DataFrame()
korea_df = pd.concat(korea_history, ignore_index=False)
korea_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,name,Ticker
Datetime,,,,,,,,,
2025-05-07 09:00:00+09:00,54500.0,55000.0,54300.0,54300.0,4209646,0.0,0.0,Samsung,005930.KS
2025-05-07 13:00:00+09:00,54350.0,54500.0,54300.0,54450.0,2880602,0.0,0.0,Samsung,005930.KS
2025-05-08 09:00:00+09:00,55100.0,55500.0,54700.0,54900.0,7738123,0.0,0.0,Samsung,005930.KS
2025-05-08 13:00:00+09:00,54900.0,55000.0,54600.0,54650.0,2831741,0.0,0.0,Samsung,005930.KS
2025-05-09 09:00:00+09:00,54700.0,54900.0,54400.0,54650.0,4073238,0.0,0.0,Samsung,005930.KS
...,...,...,...,...,...,...,...,...,...
2025-07-01 13:00:00+09:00,58200.0,58500.0,57800.0,58000.0,21460,0.0,0.0,DoubleUGames,192080.KS
2025-07-02 09:00:00+09:00,57500.0,57700.0,56700.0,56900.0,21834,0.0,0.0,DoubleUGames,192080.KS
2025-07-02 13:00:00+09:00,56900.0,58100.0,56800.0,58000.0,23933,0.0,0.0,DoubleUGames,192080.KS


In [172]:
# 인덱스 초기화 (Datetime을 일반 열로)
korea_df = korea_df.reset_index()

# 'name'과 '회사명' 기준으로 병합
korea_df = pd.merge(korea_df, df_kr, left_on='name', right_on='Name', how='left')

# 병합 후 필요 없는 중복 열 제거 (예: 회사명)
korea_df = korea_df.drop(columns=['Name'])

# 다시 Datetime을 인덱스로 설정하려면:
korea_df = korea_df.set_index('Datetime')


In [173]:
korea_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,name,Ticker_x,Rank,Ticker_y,Market Cap,Price,Today
Datetime,,,,,,,,,,,,,,
2025-05-07 09:00:00+09:00,54500.0,55000.0,54300.0,54300.0,4209646,0.0,0.0,Samsung,005930.KS,1,005930.KS,$309.70 B,$46.93,4.93%
2025-05-07 13:00:00+09:00,54350.0,54500.0,54300.0,54450.0,2880602,0.0,0.0,Samsung,005930.KS,1,005930.KS,$309.70 B,$46.93,4.93%
2025-05-08 09:00:00+09:00,55100.0,55500.0,54700.0,54900.0,7738123,0.0,0.0,Samsung,005930.KS,1,005930.KS,$309.70 B,$46.93,4.93%
2025-05-08 13:00:00+09:00,54900.0,55000.0,54600.0,54650.0,2831741,0.0,0.0,Samsung,005930.KS,1,005930.KS,$309.70 B,$46.93,4.93%
2025-05-09 09:00:00+09:00,54700.0,54900.0,54400.0,54650.0,4073238,0.0,0.0,Samsung,005930.KS,1,005930.KS,$309.70 B,$46.93,4.93%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-01 13:00:00+09:00,58200.0,58500.0,57800.0,58000.0,21460,0.0,0.0,DoubleUGames,192080.KS,100,192080.KS,$0.84 B,$42.51,0.34%
2025-07-02 09:00:00+09:00,57500.0,57700.0,56700.0,56900.0,21834,0.0,0.0,DoubleUGames,192080.KS,100,192080.KS,$0.84 B,$42.51,0.34%
2025-07-02 13:00:00+09:00,56900.0,58100.0,56800.0,58000.0,23933,0.0,0.0,DoubleUGames,192080.KS,100,192080.KS,$0.84 B,$42.51,0.34%


In [174]:
#필요한 열 선택, 보기쉽게 재배치
korea_df = korea_df[['Rank','Ticker_x', 'name', 'Open', 'High', 'Low', 'Close', 'Volume']]
korea_df.columns = ['현재 시총 순위', '종목 코드', '기업명', '시가', '고가', '저가', '종가', '거래량']
korea_df

,현재 시총 순위,종목 코드,기업명,시가,고가,저가,종가,거래량
Datetime,,,,,,,,
2025-05-07 09:00:00+09:00,1,005930.KS,Samsung,54500.0,55000.0,54300.0,54300.0,4209646
2025-05-07 13:00:00+09:00,1,005930.KS,Samsung,54350.0,54500.0,54300.0,54450.0,2880602
2025-05-08 09:00:00+09:00,1,005930.KS,Samsung,55100.0,55500.0,54700.0,54900.0,7738123
2025-05-08 13:00:00+09:00,1,005930.KS,Samsung,54900.0,55000.0,54600.0,54650.0,2831741
2025-05-09 09:00:00+09:00,1,005930.KS,Samsung,54700.0,54900.0,54400.0,54650.0,4073238
...,...,...,...,...,...,...,...,...
2025-07-01 13:00:00+09:00,100,192080.KS,DoubleUGames,58200.0,58500.0,57800.0,58000.0,21460
2025-07-02 09:00:00+09:00,100,192080.KS,DoubleUGames,57500.0,57700.0,56700.0,56900.0,21834
2025-07-02 13:00:00+09:00,100,192080.KS,DoubleUGames,56900.0,58100.0,56800.0,58000.0,23933


### 야후 API를 이용하여 두달간의 주가 저장 - 전세계

In [175]:
global_history = []

for i in range(len(df_gb)):
    name = df_gb['Name'][i]
    Ticker = df_gb['Ticker'][i]
    ticker = yf.Ticker(Ticker)
    hist = ticker.history(period="2mo", interval="4h")
    hist['name'] = name
    hist['Ticker'] = Ticker
    global_history.append(hist)

global_history


$IHC.AE: possibly delisted; no price data found  (period=2mo) (Yahoo error = "No data found, symbol may be delisted")


[                                 Open        High         Low       Close  \
 Datetime                                                                    
 2025-05-05 09:30:00-04:00  112.904999  114.379997  112.660004  114.129997   
 2025-05-05 13:30:00-04:00  114.135002  114.660004  113.669998  113.809998   
 2025-05-06 09:30:00-04:00  111.639999  113.790001  110.821999  112.919998   
 2025-05-06 13:30:00-04:00  112.930000  114.739998  112.720001  113.445000   
 2025-05-07 09:30:00-04:00  113.044998  114.220001  112.614998  113.945000   
 ...                               ...         ...         ...         ...   
 2025-06-30 13:30:00-04:00  157.824997  158.660004  156.839996  157.869995   
 2025-07-01 09:30:00-04:00  156.259995  157.199997  151.490005  154.274994   
 2025-07-01 13:30:00-04:00  154.330002  154.880005  152.910095  153.279999   
 2025-07-02 09:30:00-04:00  153.029999  157.600006  152.979996  157.205399   
 2025-07-02 13:30:00-04:00  157.205002  157.429993  156.509995  

In [176]:
len(global_history)

100

In [177]:
global_df = pd.DataFrame()
global_df = pd.concat(global_history)
global_df


,Open,High,Low,Close,Volume,Dividends,Stock Splits,name,Ticker,Adj Close
2025-05-05 09:30:00-04:00,112.904999,114.379997,112.660004,114.129997,79898207.0,0.0,0.0,NVIDIA,NVDA,NaN
2025-05-05 13:30:00-04:00,114.135002,114.660004,113.669998,113.809998,32525343.0,0.0,0.0,NVIDIA,NVDA,NaN
2025-05-06 09:30:00-04:00,111.639999,113.790001,110.821999,112.919998,91150436.0,0.0,0.0,NVIDIA,NVDA,NaN
2025-05-06 13:30:00-04:00,112.930000,114.739998,112.720001,113.445000,43956454.0,0.0,0.0,NVIDIA,NVDA,NaN
2025-05-07 09:30:00-04:00,113.044998,114.220001,112.614998,113.945000,77486400.0,0.0,0.0,NVIDIA,NVDA,NaN
...,...,...,...,...,...,...,...,...,...,...
2025-07-01 17:00:00+02:00,44.389999,44.500000,44.340000,44.480000,197375.0,0.0,0.0,Inditex,ITX.MC,NaN
2025-07-02 09:00:00+02:00,44.360001,44.490002,43.939999,44.090000,143760.0,0.0,0.0,Inditex,ITX.MC,NaN
2025-07-02 13:00:00+02:00,44.090000,44.310001,43.919998,44.040001,168242.0,0.0,0.0,Inditex,ITX.MC,NaN
2025-07-02 17:00:00+02:00,44.040001,44.070000,43.990002,44.049999,22605.0,0.0,0.0,Inditex,ITX.MC,NaN


In [178]:
# 인덱스 초기화 (Datetime을 일반 열로)
global_df = global_df.reset_index()

# 'name'과 '회사명' 기준으로 병합
global_df = pd.merge(global_df, df_gb, left_on='name', right_on='Name', how='left')

# 병합 후 필요 없는 중복 열 제거 (예: 회사명)
global_df = global_df.drop(columns=['Name'])

global_df

,index,Open,High,Low,Close,Volume,Dividends,Stock Splits,name,Ticker_x,Adj Close,Rank,Ticker_y,Market Cap,Price,Today
0,2025-05-05 09:30:00-04:00,112.904999,114.379997,112.660004,114.129997,79898207.0,0.0,0.0,NVIDIA,NVDA,NaN,1,NVDA,$3.834 T,$157.25,2.58%
1,2025-05-05 13:30:00-04:00,114.135002,114.660004,113.669998,113.809998,32525343.0,0.0,0.0,NVIDIA,NVDA,NaN,1,NVDA,$3.834 T,$157.25,2.58%
2,2025-05-06 09:30:00-04:00,111.639999,113.790001,110.821999,112.919998,91150436.0,0.0,0.0,NVIDIA,NVDA,NaN,1,NVDA,$3.834 T,$157.25,2.58%
3,2025-05-06 13:30:00-04:00,112.930000,114.739998,112.720001,113.445000,43956454.0,0.0,0.0,NVIDIA,NVDA,NaN,1,NVDA,$3.834 T,$157.25,2.58%
4,2025-05-07 09:30:00-04:00,113.044998,114.220001,112.614998,113.945000,77486400.0,0.0,0.0,NVIDIA,NVDA,NaN,1,NVDA,$3.834 T,$157.25,2.58%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8602,2025-07-01 17:00:00+02:00,44.389999,44.500000,44.340000,44.480000,197375.0,0.0,0.0,Inditex,ITX.MC,NaN,100,ITX.MC,$160.92 B,$51.66,0.48%
8603,2025-07-02 09:00:00+02:00,44.360001,44.490002,43.939999,44.090000,143760.0,0.0,0.0,Inditex,ITX.MC,NaN,100,ITX.MC,$160.92 B,$51.66,0.48%
8604,2025-07-02 13:00:00+02:00,44.090000,44.310001,43.919998,44.040001,168242.0,0.0,0.0,Inditex,ITX.MC,NaN,100,ITX.MC,$160.92 B,$51.66,0.48%
8605,2025-07-02 17:00:00+02:00,44.040001,44.070000,43.990002,44.049999,22605.0,0.0,0.0,Inditex,ITX.MC,NaN,100,ITX.MC,$160.92 B,$51.66,0.48%


In [179]:
#필요한 열 선택, 보기쉽게 재배치
global_df = global_df[['Rank','Ticker_x', 'name', 'Open', 'High', 'Low', 'Close', 'Volume']]
global_df.columns = ['현재 시총 순위', '종목 코드', '기업명', '시가', '고가', '저가', '종가', '거래량']
global_df

,현재 시총 순위,종목 코드,기업명,시가,고가,저가,종가,거래량
0,1,NVDA,NVIDIA,112.904999,114.379997,112.660004,114.129997,79898207.0
1,1,NVDA,NVIDIA,114.135002,114.660004,113.669998,113.809998,32525343.0
2,1,NVDA,NVIDIA,111.639999,113.790001,110.821999,112.919998,91150436.0
3,1,NVDA,NVIDIA,112.930000,114.739998,112.720001,113.445000,43956454.0
4,1,NVDA,NVIDIA,113.044998,114.220001,112.614998,113.945000,77486400.0
...,...,...,...,...,...,...,...,...
8602,100,ITX.MC,Inditex,44.389999,44.500000,44.340000,44.480000,197375.0
8603,100,ITX.MC,Inditex,44.360001,44.490002,43.939999,44.090000,143760.0
8604,100,ITX.MC,Inditex,44.090000,44.310001,43.919998,44.040001,168242.0
8605,100,ITX.MC,Inditex,44.040001,44.070000,43.990002,44.049999,22605.0
